# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,2.40,84,100,2.69,GS,1732566300
1,1,invercargill,-46.4000,168.3500,15.01,63,44,6.26,NZ,1732566279
2,2,puerto ayora,-0.7393,-90.3518,24.75,79,29,3.58,EC,1732565824
3,3,ribeira grande,38.5167,-28.7000,19.32,94,75,4.63,PT,1732565606
4,4,uturoa,-16.7333,-151.4333,26.67,80,95,5.40,PF,1732565867


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng","Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",  
    scale=0.1,        
    color="City"       
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions_df = city_data_df[ 
    (city_data_df['Max Temp'] < 35) & 
    (city_data_df['Max Temp'] > 15) & 
    (city_data_df['Wind Speed'] < 6) & 
    (city_data_df['Humidity'] < 80) &
    (city_data_df['Cloudiness'] < 50)
]

# Drop any rows with null values 
ideal_conditions_df = ideal_conditions_df.dropna() 

# Display sample data 
ideal_conditions_df.tail()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
581,581,port augusta,-32.5000,137.7667,23.35,67,36,0.98,AU,1732567464
583,583,kapa'a,22.0752,-159.3190,26.93,75,0,5.14,US,1732567466
589,589,ghat,24.9647,10.1728,16.87,42,0,1.42,LY,1732567473
590,590,hilo,19.7297,-155.0900,26.73,73,0,2.06,US,1732567474
594,594,baiti,13.7450,-14.7482,29.75,15,0,1.51,SN,1732567478


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_conditions_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,2,puerto ayora,-0.7393,-90.3518,24.75,79,29,3.58,EC,1732565824,
13,13,constitucion,-35.3333,-72.4167,17.68,64,0,5.43,CL,1732566177,
16,16,wailua homesteads,22.0669,-159.3780,26.18,75,0,5.14,US,1732566118,
50,50,cabo san lucas,22.8909,-109.9124,28.17,41,40,3.09,MX,1732566002,
51,51,lazaro cardenas,17.9583,-102.2000,28.44,66,0,4.03,MX,1732566408,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = radius = 10000 
params = { 
    "categories":  "accommodation.hotel", 
    "limit": 1, 
    "apiKey": geoapify_key,

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat'] 
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}" 
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
constitucion - nearest hotel: Hostal Alameda
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
cabo san lucas - nearest hotel: Comfort Rooms
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
kailua-kona - nearest hotel: PACIFIC 19 Kona
belmonte - nearest hotel: No hotel found
essaouira - nearest hotel: Essaouira Wind Palace
avarua - nearest hotel: Paradise Inn
porto novo - nearest hotel: Hôtel 6500 CFA
fort bragg - nearest hotel: Airborne Inn Lodging
palora - nearest hotel: Hotel Sol De Oriente
murrells inlet - nearest hotel: Inlet Sports Lodge
katsuren-haebaru - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
gereida - nearest hotel: No hotel found
tambura - nearest hotel: No hotel found
hafar al-batin - nearest hotel: فندق توليب بلازا
al khawr - nearest hotel: No hotel found
zouerate - nearest hotel: فندق تيرس زمور
sao jose da coroa g

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
2,2,puerto ayora,-0.7393,-90.3518,24.75,79,29,3.58,EC,1732565824,Hostal La Mirada De Solitario George
13,13,constitucion,-35.3333,-72.4167,17.68,64,0,5.43,CL,1732566177,Hostal Alameda
16,16,wailua homesteads,22.0669,-159.3780,26.18,75,0,5.14,US,1732566118,Hilton Garden Inn Kauai Wailua Bay
50,50,cabo san lucas,22.8909,-109.9124,28.17,41,40,3.09,MX,1732566002,Comfort Rooms
51,51,lazaro cardenas,17.9583,-102.2000,28.44,66,0,4.03,MX,1732566408,Hotel Sol del Pacífico
...,...,...,...,...,...,...,...,...,...,...,...
581,581,port augusta,-32.5000,137.7667,23.35,67,36,0.98,AU,1732567464,Hotel Commonwealth
583,583,kapa'a,22.0752,-159.3190,26.93,75,0,5.14,US,1732567466,Pono Kai Resort
589,589,ghat,24.9647,10.1728,16.87,42,0,1.42,LY,1732567473,فندق تاسيلي
590,590,hilo,19.7297,-155.0900,26.73,73,0,2.06,US,1732567474,Dolphin Bay Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",  # or any other relevant metric
    scale=0.5,
    color="City",
    hover_cols=['Hotel_Name', 'Country']  # Add hotel name and country to hover information
)
# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)